In [65]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from collections import ChainMap

In [66]:
#Helping functions

def get_IS_url(stock):
    url = "https://finance.yahoo.com/quote/{}/financials?p={}".format(stock, stock)
    return url

def get_BS_url(stock):  
    url = "https://finance.yahoo.com/quote/{}/balance-sheet?p={}".format(stock, stock)
    return url

def get_stockInfo_urf(stock):
    url = "https://finance.yahoo.com/quote/{}/key-statistics/".format(stock)
    return url

def get_CF_url(stock):
    url = "https://finance.yahoo.com/quote/{}/cash-flow?p={}".format(stock, stock)
    return url

def open_headless():
    options = Options()
    options.headless = True
    Driverpath = '/Users/raphael/Documents/Investment Notesbooks/chromedriver'
    driver = webdriver.Chrome(options = options, executable_path= Driverpath)
    return driver

def close_headess(driver):
    driver.quit()

def scrap_yahoo_table(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.content,'lxml')
    title = soup.select("span[data-reactid*='19']")[0].text
    table = soup.find_all('table')[0]
    df = pd.read_html(str(table))[0]
    df.columns = df.iloc[0,:]
    df.rename(columns={df.columns[0]:title}, inplace=True)
    df.set_index(df.columns[0], inplace=True)
    df.drop(df.index[0], inplace=True)
    df_tofloat = df.apply(pd.to_numeric, errors='coerce')
    return df_tofloat

def scrap_JS_yahoo_table(page):
    soup = BeautifulSoup(page.page_source,'lxml')
    title = soup.select("span[data-reactid*='19']")[0].text
    table = soup.find_all('table')[0]
    df = pd.read_html(str(table))[0]
    df.columns = df.iloc[0,:]  
    df.rename(columns={df.columns[0]:title}, inplace=True)
    df.set_index(df.columns[0], inplace=True)
    df.drop(df.index[0], inplace=True)
    df_tofloat = df.apply(pd.to_numeric, errors='coerce')
    return df_tofloat
    

def get_stock_info(stock):
    url = get_stockInfo_urf(stock)
    res = requests.get(url)
    soup = BeautifulSoup(res.content,'lxml')
    table = soup.find_all('table',{"class":"table-qsp-stats"})
    df = pd.read_html(str(table))[0]
    df.columns = ['Label', 'Value'] 
    df.set_index('Label', inplace=True)
    df = df.iloc[0:2]
    df.loc['Market Cap (intraday) 5'] = df.loc['Market Cap (intraday) 5'].str.replace('B', '', regex=False)
    df.loc['Enterprise Value 3'] = df.loc['Enterprise Value 3'].str.replace('B', '', regex=False)
    df_tofloat = df.apply(pd.to_numeric, errors='coerce')
    df_tofloat.loc['Market Cap (intraday) 5'][0] = df_tofloat.loc['Market Cap (intraday) 5'][0] * 1000000
    df_tofloat.loc['Enterprise Value 3'][0] = df_tofloat.loc['Enterprise Value 3'][0] * 1000000
    return df_tofloat



In [67]:
#Get Fundamentals
def get_income_statement(stock):    
    url = get_IS_url(stock)
    table = scrap_yahoo_table(url)
    return table

def get_balance_sheet(stock):

    url = get_BS_url(stock)
    table = scrap_yahoo_table(url)
    return table

def get_cashflow(stock):
    url = get_CF_url(stock)
    table = scrap_yahoo_table(url)
    return table

In [68]:
# Trailing fundamentals
def get_trailing_IS(stock):
    driver = open_headless()
    url = get_IS_url(stock)
    driver.get(url)
    xpath = "//*[@id='Col1-1-Financials-Proxy']/section/div[1]/div[2]/button"
    python_button = driver.find_element_by_xpath(xpath) #FHSU
    python_button.click()
    driver.implicitly_wait(30)
    df = scrap_JS_yahoo_table(driver)
    df['trailing year'] = df.sum(axis=1) 
    close_headess(driver)
    return df

def get_trailing_BS(stock):
    driver = open_headless()
    url = get_BS_url(stock)
    driver.get(url)
    xpath = "//*[@id='Col1-1-Financials-Proxy']/section/div[1]/div[2]/button"
    python_button = driver.find_element_by_xpath(xpath)
    python_button.click()
    driver.implicitly_wait(30)
    df = scrap_JS_yahoo_table(driver)
    df['trailing year'] = df.sum(axis=1)
    close_headess(driver)
    return df

def get_trailing_cf(stock):
    driver = open_headless()
    url = get_CF_url(stock)
    driver.get(url)
    xpath = "//*[@id='Col1-1-Financials-Proxy']/section/div[1]/div[2]/button"
    python_button = driver.find_element_by_xpath(xpath)
    python_button.click()
    driver.implicitly_wait(10)
    df = scrap_JS_yahoo_table(driver)
    df['trailing year'] = df.sum(axis=1)
    close_headess(driver)
    return df

In [69]:
# Get Magic Formula
def get_ROC(stock):
    stock_BS = get_trailing_BS(stock)
    stock_IS = get_trailing_IS(stock)
    EBIT = stock_IS.loc['Earnings Before Interest and Taxes']
    Net_working_capital = stock_BS.loc['Total Current Assets'] - stock_BS.loc['Cash And Cash Equivalents'] - stock_BS.loc['Total Current Liabilities']
    Net_Fixed_asset = stock_BS.loc['Property Plant and Equipment']

    ROC = EBIT/(Net_working_capital + Net_Fixed_asset)
    return ROC

def get_earning_yield(stock):
    stock_IS = get_trailing_IS(stock)
    stock_info = get_stock_info(stock)
    EBIT = stock_IS.loc['Earnings Before Interest and Taxes']
    Entreprise_value = stock_info.loc['Enterprise Value 3'][0]
    #print(Entreprise_value + EBIT)
    #print(type(Entreprise_value))
    ey = EBIT/Entreprise_value
    return ey

def get_magic_formula(stocks_list):
    stock_KPI = {}
    print(stocks_list)
    for stocks in stocks_list:
        ey = get_earning_yield(stocks).loc['trailing year']
        roc = get_ROC(stocks).loc['trailing year']
        stock_KPI.update({stocks:[ey,roc]})
        print(stock_KPI)
    mf_df = pd.DataFrame.from_dict(stock_KPI).T
    mf_df.columns = ['EY', 'ROC']
    mf_df['rankEY'] = mf_df['EY'].rank(ascending = False)
    mf_df['rankROC'] = mf_df['ROC'].rank(ascending = False)
    mf_df['Magic Formula'] = mf_df['rankEY'] + mf_df['rankROC']
    return mf_df

In [71]:
x = get_balance_sheet("CRTO")
x

,12/31/2018,12/31/2017,12/31/2016,12/31/2015
Balance Sheet,,,,
Current Assets,NaN,NaN,NaN,NaN
Cash And Cash Equivalents,NaN,414111.0,270317.0,353537.0
Short Term Investments,NaN,NaN,NaN,NaN
Net Receivables,551371.0,557023.0,456190.0,294914.0
Inventory,NaN,NaN,NaN,NaN
Other Current Assets,5759.0,8403.0,2439.0,5488.0
Total Current Assets,NaN,991767.0,742584.0,663414.0
Long Term Investments,14060.0,19525.0,17029.0,17184.0
Property Plant and Equipment,NaN,161738.0,108581.0,82482.0
